In [2]:
import pandas as pd
import json

In [ ]:
# Charger les fichiers JSON en DataFrames
with open("fichier_1.json", "r", encoding="utf-8") as file1:
    df1 = pd.DataFrame(json.load(file1))

with open("fichier_2.json", "r", encoding="utf-8") as file2:
    df2 = pd.DataFrame(json.load(file2))

# Trouver les lignes où la_street de df1 correspond à or_street de df2
matching_rows = df1[df1['la_street'].isin(df2['or_street'])]

# Afficher les résultats
print(matching_rows)

# Optionnel : Enregistrer les résultats dans un fichier JSON
matching_rows.to_json("matching_addresses.json", orient="records", force_ascii=False, indent=4)

print("Les correspondances ont été enregistrées dans 'matching_addresses.json'")

In [ ]:
import pandas as pd
import json
import re

# Fonction de normalisation des adresses
def normalize_address(address):
    return re.sub(r'[^a-zA-Z0-9 .\-()]', '', address).lower().strip()

# Charger les fichiers JSON en DataFrames
with open("logistic_address_20250213_084740.json", "r", encoding="utf-8") as file1:
    df1 = pd.DataFrame(json.load(file1))

with open("companies_20250210_165746.json", "r", encoding="utf-8") as file2:
    df2 = pd.DataFrame(json.load(file2))

# Appliquer la normalisation aux colonnes concernées
df1['normalized_la_street'] = df1['la_street'].apply(normalize_address)
df2['normalized_or_street'] = df2['or_street'].apply(normalize_address)

# Trouver les lignes où la_street de df1 correspond à or_street de df2
matching_rows = df1[df1['normalized_la_street'].isin(df2['normalized_or_street'])]

# Afficher les résultats
print(matching_rows)

# Optionnel : Enregistrer les résultats dans un fichier JSON
matching_rows.to_json("matching_addresses.json", orient="records", force_ascii=False, indent=4)

print("Les correspondances ont été enregistrées dans 'matching_addresses.json'")